In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [90.7 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Ge

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-08-16 15:09:00--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2022-08-16 15:09:00 (9.30 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("TheReason").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
#sc = SparkContext('local')
#spark = SparkSession(sc)

from pyspark import SparkFiles
# Import data types
from pyspark.sql.types import *

url = "https://s3.amazonaws.com/the.chum.bucket/USvideos.csv"
spark.sparkContext.addFile(url)

schema = StructType(
   [StructField('video_id', StringType(), True),
    StructField('trending_date', StringType(), True),
    StructField('title', StringType(), True),
    StructField('channel_title', StringType(), True),
    StructField('category_id', IntegerType(), True),
    StructField('publish_time', TimestampType(), True),
    StructField('tags', StringType(), True),
    StructField('views', IntegerType(), True),
    StructField('likes', IntegerType(), True),
    StructField('dislikes', IntegerType(), True),
    StructField('comment_count', IntegerType(), True),
    StructField('thumbnail_link', StringType(), True),
    StructField('comments_disabled', BooleanType(), True),
    StructField('ratings_disabled', BooleanType(), True),
    StructField('video_error_or_removed', BooleanType(), True),
    StructField('description', StringType(), True)
   ]
  )

df = spark.read.option("header", "true").schema(schema).csv(SparkFiles.get(""), header=True)

df.show()

+-----------+-------------+--------------------+--------------------+-----------+-------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|   video_id|trending_date|               title|       channel_title|category_id|       publish_time|                tags|  views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+-----------+-------------+--------------------+--------------------+-----------+-------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|2kyS6SvSYSE|     17.14.11|WE WANT TO TALK A...|        CaseyNeistat|         22|2017-11-13 17:13:01|     SHANtell martin| 748374| 57527|    2966|        15954|https://i.ytimg.c...|            false|           false| 

In [ ]:
trending_df = df.select(["category_id", "views", "likes", "dislikes", "comment_count", "channel_title", "publish_time", "trending_date"])
trending_df.show()

+-----------+-------+------+--------+-------------+--------------------+-------------------+-------------+
|category_id|  views| likes|dislikes|comment_count|       channel_title|       publish_time|trending_date|
+-----------+-------+------+--------+-------------+--------------------+-------------------+-------------+
|         22| 748374| 57527|    2966|        15954|        CaseyNeistat|2017-11-13 17:13:01|     17.14.11|
|         24|2418783| 97185|    6146|        12703|     LastWeekTonight|2017-11-13 07:30:00|     17.14.11|
|         23|3191434|146033|    5339|         8181|        Rudy Mancuso|2017-11-12 19:05:24|     17.14.11|
|         24| 343168| 10172|     666|         2146|Good Mythical Mor...|2017-11-13 11:00:04|     17.14.11|
|         24|2095731|132235|    1989|        17518|            nigahiga|2017-11-12 18:01:41|     17.14.11|
|         28| 119180|  9763|     511|         1434|            iJustine|2017-11-13 19:07:23|     17.14.11|
|         24|2103417| 15993|    2445|

In [ ]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password')

# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://the-reason.cklnwcyuqruo.us-east-1.rds.amazonaws.com/postgres"
config = {"user":"postgres",
          "password": password,
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [34]:
# Write trending_df to table in RDS
trending_df.write.jdbc(url=jdbc_url, table='youtube_analysis', mode=mode, properties=config)